In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format  # set other global format

<b> Reading the file </b>

In [2]:
pd.set_option('display.max_colwidth', None) 
print('Reading the Trade Atlas Indonesia File')
df=pd.read_excel("C:/Users/DrC/Desktop/Eureka Idea Co/Number of Test Testing File/Indonesia 2021-Including Exchange Rates WA V2_Updated with Verification.xlsx",sheet_name='Indonesia 2021-Indonesia 2021V1')

Reading the Trade Atlas Indonesia File


<b>Data cleaning </b>
- Removing starting and trailing spaces
- Removing the identified terms
- Removing punctuation
- Removing double spaces
- Adding Keywords as columns in the dataframe. These columns are used as indicators/boolean checks for the presence of keywords.

In [3]:
df1=df
print('Preparing Product Details Column for the extraction of number of tests')
#Adding the keywards to the pandas dataframe
Keyword_list=['T','TES','TEST','BOX','CASSETTES','RXN','RNX','REACTION','KIT','PCS','PCE','STRIP']
df1 = df1.reindex(columns = df1.columns.tolist() + Keyword_list) #Adding empty columns to the python list 

#PDC CLEANING
df1['PDC']=df['PRODUCT DETAILS'] #NEW COLUMN PDC (Product Details Cleaning) has the the product detail data in it now.

#Removing starting and trailing spaces
df1['PDC']= df1['PDC'].str.strip() #

#Remove COVID-19 OR COVID 19 from the string
df1['PDC']= df1['PDC'].str.replace('COVID-19', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace('COVID 19', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace(' COVID 19 ', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace('COVID -19', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace('COVID- 19', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('COVID TEST 19', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('QUANT 20', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('POCKIT', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('1KIT', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('SCNTH', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('SCNTL', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('DETECTION KIT', ' ', case = False) 
df1['PDC']= df1['PDC'].str.replace('EAR99', ' ', case = False) 

 
#Remove COV-2 related keywords from the string
df1['PDC']= df1['PDC'].str.replace('COV-2', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace('COV 2', ' ', case = False)
df1['PDC']= df1['PDC'].str.replace('COV2', ' ', case = False)


#Remove VAR-2 from the string
df1['PDC']= df1['PDC'].str.replace('VAR2', ' ', case = False) #SOLVES THE ISSUE 

#Remove SARS COV 2 related keywords from the string
df1['PDC']= df1['PDC'].str.replace('SARS COV 2', ' ', case = False) #SOLVES THE ISSUE 
df1['PDC']= df1['PDC'].str.replace('SARS-COV-2', ' ', case = False) #SOLVES THE ISSUE 
df1['PDC']= df1['PDC'].str.replace('SARSCOV3', ' ', case = False) #SOLVES THE ISSUE 

#Remove the punctuation and the brackets. 
df1['PDC'] = df1['PDC'].str.replace(r'[^\w\s]+', '') 

#Removing double spaces between the words
df1['PDC']= df1['PDC'].replace('\s+', ' ', regex=True)



Preparing Product Details Column for the extraction of number of tests


<b> Checking presence of keywords in every row of product details </b>

In [4]:
#Checking presence of keywords in every row of product details  
for x in range(0,len(df1['PDC'])):
    for y in range(0,len(Keyword_list)):
        if(Keyword_list[y]!='T'):
            if(Keyword_list[y] in df1['PDC'][x]):
                df1[Keyword_list[y]][x]=1
            else:
                df1[Keyword_list[y]][x]=0


<b> Splitting the product detail column in to words </b>

In [5]:
print('Processing Product Details Column for the extraction of number of tests')

#Splitting the product detail column in to words
df1['PDC_SPLIT']=df1['PDC'].str.split()

Processing Product Details Column for the extraction of number of tests


<b> Checking the presence of T </b>

In [6]:
#Checking the presence of T in the split words
for x in range(0,len(df1['PDC'])):
    for y in range(0,len(df1['PDC_SPLIT'][x])):
        if(df1['PDC_SPLIT'][x][y]=='T'):
            df1['T'][x]=1


<b>Checking if the word contains a digit</b>

In [7]:
def containsNumber(value):
    for character in value:
        if character.isdigit():
            return True
    return False

<b>Creating a column "words" that has words from the product detail split column containing digits or a combination of digits or characters.  </b>

In [8]:
#Creating a column with an empty list
df1['words'] = np.empty((len(df1), 0)).tolist()
#Taking the values from the PDC split that has numbers only. 
for i in range(0,len(df1['PDC_SPLIT'])):
    for w in range(0,len(df1['PDC_SPLIT'][i])):
    
        if(containsNumber(df1['PDC_SPLIT'][i][w])==True):#if conition if the word contains a digit
            df1['words'][i].append(df1['PDC_SPLIT'][i][w]) #appending the list with words that have number in it



<b>A new column is generated called refined words. Words that gets in to this column meets the following conditions:</b> <br>
1.There is a digit present and the number is than 2500 <br>
2. The keyword is present with the number.  


In [9]:
df1['refined_words'] = np.empty((len(df1), 0)).tolist()
#selecting only the words that contain numbers
for x in range(0,len(df1['words'])):
    for y in range(0,len(df1['words'][x])):
        if((df1['words'][x][y].isdigit() and int(df1['words'][x][y])<=2500)  or ('T' in df1['words'][x][y]) or ('RXN' in df1['words'][x][y]) or ('PCE' in df1['words'][x][y]) or ('EA' in df1['words'][x][y]) or ('CASSE' in df1['words'][x][y]) or ('BOX' in df1['words'][x][y]) or ('PCS' in df1['words'][x][y]) or ('KIT' in df1['words'][x][y]) or ('TES' in df1['words'][x][y]) or ('STRIP' in df1['words'][x][y])):
            df1['refined_words'][x].append(df1['words'][x][y]) 
    


<b>A Column for Manual Verification</b>

In [10]:
df1['REQUIRES_MANUAL_VERIFICATION']=''

<b>Following conditions are implemented in the code below: </b>
1) if the refined words only contain the numbers then check the range of the numbers. If it is >=5 and <=2500 then we take it as valid test. <br> 
2) if the refine words column contain numbers only and all are equal, then we remove the duplicates and get one value only. <br> 
3) If the number of refined words column has one value and contains digit with characters then check if it has a keyword attached to it. If yes, then take that number. <Br> 


In [11]:
import re
df1['test_quantity'] = np.empty((len(df1), 0)).tolist()
 
df1=df1.fillna(0)
for x in range(0,len(df1['refined_words'])):
    for y in range(0,len(df1['refined_words'][x])):
        res=all(map(str.isdigit, df1['refined_words'][x])) #all the elements in the list are digit only- return true
        if(res==True and len(df1['refined_words'][x])!=0):  #checking that the list contain all the numbers only
            if(int(df1['refined_words'][x][y])>=5 and int(df1['refined_words'][x][y])<=2500): #Sanity range taken to be greater than 5 and less than 2500
                df1['test_quantity'][x].append(re.sub("[^0-9]", "", df1['refined_words'][x][y]))
        result_equal = all(element == df1['refined_words'][x][0] for element in df1['refined_words'][x]) #if all the elements in the list are equal
        if(result_equal==True and len(df1['refined_words'][x])>1 and res==True): #only digits lists being handled uptil now
            df1['test_quantity'][x]=np.unique(np.array(df1['refined_words'][x])).tolist() #converting to list back again 
     
        
        #list contains only one value and that value is a number with characters
        elif(len(df1['refined_words'][x])==1 and int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))>5 and int(re.sub("[^0-9]", "", df1['refined_words'][x][y])) and res==False):  #if length is one then remove the characters and insert the digits
         
            if(('T' in df1['refined_words'][x][y])):
                count_t = df1['refined_words'][x][y].count('T')
                if(count_t==1 or count_t==2 or count_t==3 or count_t==4):
                    df1['T'][x]=1   #This T condition is just for the boolean check
                    df1['test_quantity'][x].append(re.sub("[^0-9]", "", df1['refined_words'][x][y])) 
                    
            else:
                df1['test_quantity'][x].append(re.sub("[^0-9]", "", df1['refined_words'][x][y]))

              
        elif(len(df1['refined_words'][x])>1 and res==False): #if any of the word contains the keyword then we need that value only.
            if((('TE' in df1['refined_words'][x][y]) or ('KIT' in df1['refined_words'][x][y]) or ('RXN' in df1['refined_words'][x][y]) or ('PCS' in df1['refined_words'][x][y]) or ('PCE' in df1['refined_words'][x][y]) or ('CASSE' in df1['refined_words'][x][y]) or ('BOX' in df1['refined_words'][x][y]) or ('RNX' in df1['refined_words'][x][y]) or ('STRIP' in df1['refined_words'][x][y]) ) and ((int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))>=5) and int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))<=2500)):
                if(int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))>=5 and int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))<=2500):
                    df1['test_quantity'][x].append(re.sub("[^0-9]", "", df1['refined_words'][x][y]))
           
            
            
            elif(('T' in df1['refined_words'][x][y])):
             
                count_t = df1['refined_words'][x][y].count('T')
                if(count_t==1): #not taking count t==2 because there are more than one words and it may cause confusion
                    if(int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))>=5 and int(re.sub("[^0-9]", "", df1['refined_words'][x][y]))<=2500):
                        df1['test_quantity'][x].append(re.sub("[^0-9]", "", df1['refined_words'][x][y])) 
                        df1['T'][x]=1
                        
print('Product Details column processing completed')

Product Details column processing completed


<b> Boolean Checks / Indicators </b>

In [12]:
print('Creating Boolean checks for the presence of Keywords')

r = re.compile(r'([0-9]+X[0-9]+T+|[0-9]+\*[0-9]+T|[0-9]+\*[0-9]+ T|[0-9]+X[0-9]+ T+)') #Regex for identifying Multiplication values
#Creating Keywords boolean checks.
df1['test_quantity_final'] = np.empty((len(df1), 0)).tolist()
df1['Auto_Data_Quality_index']=""
boolean_counter=0
for x in range(0,len(df1['refined_words'])): #can use any column just need to get the length
    for y in range(0,len(Keyword_list)):
        if (df1[Keyword_list[y]][x]==1):
             boolean_counter=boolean_counter+1
            
    
    if(boolean_counter>=1 and len(df1['test_quantity'][x])==1):
        
        df1['test_quantity_final'][x]=int(df1['test_quantity'][x][0])
        df1['Auto_Data_Quality_index'][x]='Keyword and Value both are present'
    
    if(boolean_counter>=1 and len(df1['test_quantity'][x])>1):
        df1['test_quantity_final'][x]=np.nan
        df1['Auto_Data_Quality_index'][x]='Keyword and Two Values are present'
        df1['REQUIRES_MANUAL_VERIFICATION'][x]=1
        
    if(boolean_counter==0):
        df1['test_quantity_final'][x]=1
        df1['Auto_Data_Quality_index'][x]='No Keyword present'
        
    if(boolean_counter>=1 and len(df1['test_quantity'][x])==0):
        df1['test_quantity_final'][x]=1
        df1['Auto_Data_Quality_index'][x]='Keyword found but no value'
    #This condition shall stay in the last
    if(df1['test_quantity_final'][x]>2500):
        df1['test_quantity_final'][x]=1
    boolean_counter=0
    
    if(r.search(df1['PRODUCT DETAILS'][x])):
        df1['test_quantity_final'][x]=np.nan
        df1['Auto_Data_Quality_index'][x]='Multiplication Values'
        df1['REQUIRES_MANUAL_VERIFICATION'][x]=1

print('Boolean checks created')
print('Number of tests extracted from Product details')
        


Creating Boolean checks for the presence of Keywords
Boolean checks created
Number of tests extracted from Product details


In [13]:
#Value counts for the Auto Data Quality Index 
df1['Auto_Data_Quality_index'].value_counts()
print(df1['Auto_Data_Quality_index'].value_counts())
print('Auto Data Quality index created')

No Keyword present                    710
Keyword found but no value            603
Keyword and Value both are present    479
Keyword and Two Values are present     12
Multiplication Values                   8
Name: Auto_Data_Quality_index, dtype: int64
Auto Data Quality index created


# NEW TASKS TO DO 

1) Categorize the product detail in to RDT AND NON RDT.  The keywords are in the methodology file. We are concerned for RDT (Antigen & Antibody). There should a column which specifies the product detail is RDT or Non RDT. <BR>
2) Convert the currency in to USD (based on specific dates) and multiply the import value.<BR>
3) We need to get the top 80% data for the RDT. Arrange the data in descending order and then get the top 80%. (Percentile) <BR>
4) Filter calibration unit and control units

<B>1) Categorize the product detail in to RDT AND NON RDT. </B>

a) Filter All product Details that have "Pcr" in it and Marks them as "Non-RDT". Mark them as "Last Data Quality Step" = 1 <br>
b) Filter All product Details that have "Antigen" in it and Marks them as "SARS-CoV-2 Antigen Rapid Diagnostic Tests" in the "Test Type" Column.<br>
c) Filter All product Details that have " Ag ", "Ag " and " Ag" in it and Marks them as "SARS-CoV-2 Antigen Rapid Diagnostic Tests" in the "Test Type" Column.<br>
d) Filter All product Details that have "antibod" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>
e) Filter All product Details that have "igg" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>
f) Filter All product Details that have "igm" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>
g) Filter "Blanks" in test types. We are not sure about these products. Primarily we can assume that these are also Non-RDT, Non-PCR, Mark them as "Non-RDT, Non PCR" in the test Type<br>
h) Now select "SARS-CoV-2 Antigen Rapid Diagnostic Tests" and "SARS-CoV-2 Antibody Rapid Diagnostic Tests" from Test Types. These are the only considerable products that require manual verification for data Quality Index.<br>
i) Filter Product Details with "CAL " and mark them as "Calibration Unit" <br>
j) Filter "Number of Tests/Box" column with "2*XXX" and Similar values and mark them as "Calibration + Test Kit" Put value in Manual Entry Number of tests/Box. For 2x100, put 100 (i.e. 100 tests and 100 Calibration unit)<br>
k) Filter Product Details with "CTL " and mark them as "Control Unit" <br>
l) Filter "Number of Tests/Box" column with "2*XXX" and Similar values and mark them as "Calibration + Test Kit" Put value in Manual Entry Number of tests/Box. For 2x100, put 100 (i.e. 100 tests and 100 Control unit)<br>
m) Filter Product Details with "CONT" and mark them as "Control Unit"<br>
n) Filter Product Details with Text Filter Contains "Quant*CO" and mark them as "Control Unit" <br>
o) Filter Product Details with Text Filter Contains "Quant*CA" and mark them as "Calibration Unit" <br>
p) Filter with "MGM" in the "Quantity Unit" Column and Mark them as Non-RDT  <br>

In [14]:
df1['Test_Type'] = np.empty((len(df1), 0)).tolist()

In [15]:
#a) Filter All product Details that have "Pcr" in it and Marks them as "Non-RDT". Mark them as "Last Data Quality Step" = 1 
#m) Filter Product Details with "CONT" and mark them as "Control Unit"
#n) Filter Product Details with Text Filter Contains "QuantCO" and mark them as "Control Unit"
#o) Filter Product Details with Text Filter Contains "QuantCA" and mark them as "Calibration Unit"

print('Labelling Non RDT products')
non_rdt=['PCR', 'CONT','CAL','CTL','QUANTCO','QUANTCA','QUANT CA', 'QUANT CO']

for x in range(0,len(df1['PRODUCT DETAILS'])):
    for y in range(0,len(non_rdt)):
        if(non_rdt[y] in df1['PRODUCT DETAILS'][x]):
            df1['Test_Type'][x].append('Non-RDT') 
    
    df1['Test_Type'][x]=np.unique(np.array(df1['Test_Type'][x])).tolist()
         

Labelling Non RDT products


In [16]:
# b) Filter All product Details that have "Antigen" in it and Marks them as "SARS-CoV-2 Antigen Rapid Diagnostic Tests" in the "Test Type" Column.
# c) Filter All product Details that have " Ag ", "Ag " and " Ag" in it and Marks them as "SARS-CoV-2 Antigen Rapid Diagnostic Tests" in the "Test Type" Column.

print('Labelling Antigen products')

antigen = ["ANTIGEN", " AG ", "AG ", " AG"]


#Performing the Boolean check
for x in range(0,len(df1['PRODUCT DETAILS'])):
    for y in range(0,len(antigen)):
        if(antigen[y] in df1['PRODUCT DETAILS'][x]):
            df1['Test_Type'][x].append('SARS-CoV-2 Antigen Rapid Diagnostic Tests') 
    
    df1['Test_Type'][x]=np.unique(np.array(df1['Test_Type'][x])).tolist()

            
            

Labelling Antigen products


In [17]:
#Filter All product Details that have "antibod" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>
#Filter All product Details that have "igg" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>
#Filter All product Details that have "igm" in it and Marks them as "SARS-CoV-2 Antibody Rapid Diagnostic Tests" in the "Test Type" Column.<br>


print('Labelling Antibody products')

antibody=['ANTIBOD','IGG', 'IGM']

#Performing the Boolean check
for x in range(0,len(df1['PRODUCT DETAILS'])):
    for y in range(0,len(antibody)):
        if(antibody[y] in df1['PRODUCT DETAILS'][x]):
            df1['Test_Type'][x].append('SARS-CoV-2 Antibody Rapid Diagnostic Tests') 
    
    df1['Test_Type'][x]=np.unique(np.array(df1['Test_Type'][x])).tolist()



Labelling Antibody products


In [18]:
#Filter "Blanks" in test types. We are not sure about these products. 
#Primarily we can assume that these are also Non-RDT, Non-PCR, Mark them as "Non-RDT, Non PCR" in the test Type


print('Labelling Non RDT and Non PCR')

for x in range(0,len(df1['Test_Type'])):
    if(len(df1['Test_Type'][x])==0):
        df1['Test_Type'][x].append('Non-RDT, Non PCR')    
 


Labelling Non RDT and Non PCR


In [19]:
#removing the list brackets
df1['Test_Type'] = df1['Test_Type'].str[0]

In [20]:
df1['Test_Type']

0                                 Non-RDT, Non PCR
1                                 Non-RDT, Non PCR
2                                          Non-RDT
3                                 Non-RDT, Non PCR
4                                 Non-RDT, Non PCR
                           ...                    
1807     SARS-CoV-2 Antigen Rapid Diagnostic Tests
1808    SARS-CoV-2 Antibody Rapid Diagnostic Tests
1809     SARS-CoV-2 Antigen Rapid Diagnostic Tests
1810                              Non-RDT, Non PCR
1811                                       Non-RDT
Name: Test_Type, Length: 1812, dtype: object

<b>Missing</b> <br> 
1) Filter "Number of Tests/Box" column with "2*XXX" and Similar values and mark them as "Calibration + Test Kit" Put value in Manual Entry Number of tests/Box. For 2x100, put 100 (i.e. 100 tests and 100 Calibration unit)<br>
2) Filter "Number of Tests/Box" column with "2*XXX" and Similar values and mark them as "Calibration + Test Kit" Put value in Manual Entry Number of tests/Box. For 2x100, put 100 (i.e. 100 tests and 100 Control unit)

# h) Now select "SARS-CoV-2 Antigen Rapid Diagnostic Tests" and "SARS-CoV-2 Antibody Rapid Diagnostic Tests" from Test Types. These are the only considerable products that require manual verification for data Quality Index. <BR>
    
 ASK THIS: p) Filter with "MGM" in the "Quantity Unit" Column and Mark them as Non-RDT

# Converting the Price Value in to USD

In [21]:
#Reading the exchange rate sheet
print('Reading the Exchange Rate file')
er=pd.read_excel("C:/Users/DrC/Desktop/Eureka Idea Co/Number of Test Testing File/Indonesia 2021-Including Exchange Rates WA V2_Updated with Verification.xlsx",sheet_name='Exchange rates')
er=er.iloc[:,0:10]



Reading the Exchange Rate file


In [22]:
er.head(5)

,Date,USD,IDR,JPY,CNY,AUD,GBP,EUR,SGD,VND
0,2020-03-16,1.00,0.00,0.01,0.14,0.61,1.23,1.12,0.70,0.00
1,2020-03-17,1.00,0.00,0.01,0.14,0.60,1.21,1.10,0.70,0.00
2,2020-03-18,1.00,0.00,0.01,0.14,0.59,1.19,1.09,0.69,0.00
3,2020-03-19,1.00,0.00,0.01,0.14,0.58,1.16,1.08,0.69,0.00
4,2020-03-20,1.00,0.00,0.01,0.14,0.59,1.18,1.07,0.69,0.00


In [23]:
#Converting datetime to date 
df1['Date'] = pd.to_datetime(df1['ARRIVAL DATE']).dt.date

In [24]:
df1['Date']=df1['Date'].astype('datetime64[ns]')

In [25]:
df1 = df1.merge(er, on='Date', how='left')

In [26]:
#CURRENCY CONVERSION

print('Converting Import Value CIF to USD')
df1["IMPORT_VALUE_CIF_USD"] = np.NaN
df1['IMPORT VALUE CIF']=df1['IMPORT VALUE CIF'].astype(float)

for x in range(0,len(df1['IMPORT VALUE CIF'])):
    if(df1['CURRENCY'][x]=='USD'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['USD'][x]
        
    elif(df1['CURRENCY'][x]=='IDR'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['IDR'][x]
        
    elif(df1['CURRENCY'][x]=='JPY'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['JPY'][x]
        
    elif(df1['CURRENCY'][x]=='CNY'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['CNY'][x]
       
    elif(df1['CURRENCY'][x]=='AUD'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['AUD'][x]
       
    elif(df1['CURRENCY'][x]=='GBP'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['GBP'][x]
       
    elif(df1['CURRENCY'][x]=='EUR'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['EUR'][x]
       
    elif(df1['CURRENCY'][x]=='SGD'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['SGD'][x]
        
    elif(df1['CURRENCY'][x]=='VND'):
        df1['IMPORT_VALUE_CIF_USD'][x]=df1['IMPORT VALUE CIF'][x]*df1['VND'][x]
    
    else:
        df1['IMPORT_VALUE_CIF_USD'][x]='unidentified currency'
       



Converting Import Value CIF to USD


In [27]:
#print(df1['IMPORT_VALUE_CIF_USD'].head(5))

# VOLUME CALCULATIONS

1) Volume1-PCE: Directly pulling the data from quantity column - Assume it is all PCES FOR NOW <BR>
2) Volume2-BOX: Quantity * Number of tests 

3) -ROUND IT TO TWO DECIMAL VALUES

4)We have two volumes. Now  we need to check which one is correct based ond
- Per Unit Weight
- Per Unit Price


5) PER UNIT GROSS WEIGHT

Divide the Gross Weight / Volume1 <BR>
Divide the Gross Weight / Volume2 <BR>

6) PER UNIT NET WEIGHT<BR>
Divide the NET Weight / Volume1<BR>
Divide the NET Weight / Volume2<BR>


7) PER UNIT PRICE<BR>
DIVIDE THE IMPORT VALUE CIF (CONVERTED TO USD) / VOLUME1 <BR>
DIVIDE THE IMPORT VALUE CIF (CONVERTED TO USD) / VOLUME2 <BR>


8) Now we need to decide which volume do we need to consider. If we don't get anyone is correct
then we have to go the manual check. 

9) WEIGHT RANGES: <BR>
RANGE: 0.01 TO 0.03 - THIS IS THE NET WEIGHT <BR>
RANGE: 0.01 TO 0.1 - THIS IS FOR THE GROSS WEIGHT<BR>


10) PRICE
RANGE: SHOULD BE <= TO $12 - one range only.

put a count  - if you get 3 match we have 75% confidence
we can fine tune this later
put a count  - if you get 2 match we have 75% confidence
put a count  - if you get 1 match we have 75% confidence

Those which have 0 matches: these are put for the next stage now...

For 100% WE WILL HAVE TO MATCH WITH THE MRL DATA. STRATEGY NEEDS TO BE DEVISED. 


In [28]:
#1) Volume1-PCE: Directly pulling the data from quantity column - Assume it is all PCES FOR NOW
#2) Volume2-BOX: Quantity * Number of tests
print('Processing Volume Calculations: ')


df1['test_quantity_final']=df1['test_quantity_final'].astype(float)
df1['VOLUME1_PCE']=df1['QUANTITY']
df1['VOLUME2_BOX']=df1['QUANTITY']*df1['test_quantity_final']

Processing Volume Calculations: 


In [29]:
#PER UNIT GROSS WEIGHT

print('PER UNIT GROSS WEIGHT - Volume1')
print('PER UNIT GROSS WEIGHT - Volume2')

#Divide the Gross Weight / Volume1
df1['PER_PCE_GROSS_WEIGHT'] = df1['GROSS WEIGHT']/df1['VOLUME1_PCE']
#Divide the Gross Weight / Volume2

df1['PER_BOX_GROSS_WEIGHT'] = df1['GROSS WEIGHT']/df1['VOLUME2_BOX']

PER UNIT GROSS WEIGHT - Volume1
PER UNIT GROSS WEIGHT - Volume2


In [30]:
#PER UNIT NET WEIGHT
print('PER UNIT NET WEIGHT - Volume1')
print('PER UNIT NET WEIGHT - Volume2')

#Divide the NET Weight / Volume1
df1['PER_PCE_NET_WEIGHT'] = df1['NET WEIGHT']/df1['VOLUME1_PCE']

#Divide the NET Weight / Volume2
df1['PER_BOX_NET_WEIGHT'] = df1['NET WEIGHT']/df1['VOLUME2_BOX']

PER UNIT NET WEIGHT - Volume1
PER UNIT NET WEIGHT - Volume2


In [31]:
#7) PER UNIT PRICE
print('PER UNIT PRICE - Volume1')
print('PER UNIT PRICE - Volume2')

#DIVIDE THE IMPORT VALUE CIF (CONVERTED TO USD) / VOLUME1
df1['PER_PCE_UNIT_PRICE']=df1['IMPORT_VALUE_CIF_USD']/df1['VOLUME1_PCE']

#DIVIDE THE IMPORT VALUE CIF (CONVERTED TO USD) / VOLUME2
df1['PER_BOX_UNIT_PRICE']=df1['IMPORT_VALUE_CIF_USD']/df1['VOLUME2_BOX']

PER UNIT PRICE - Volume1
PER UNIT PRICE - Volume2


In [32]:
#WEIGHT RANGES:
#RANGE: 0.01 TO 0.03 - THIS IS THE NET WEIGHT

print('Creating Boolean checks for Net weight, Gross weight and Unit price - by Volume1 and Volume 2')
df1['NEW_QUANTITY_UNIT'] = np.empty((len(df1), 0)).tolist()

df1['EQUAL_VOLUMES']=''
df1['PCE_NET_WEIGHT_CHECK']=0
df1['PCE_GROSS_WEIGHT_CHECK']=0

df1['BOX_NET_WEIGHT_CHECK']=0
df1['BOX_GROSS_WEIGHT_CHECK']=0

df1['PCE_PRICE_CHECK']=0
df1['BOX_PRICE_CHECK']=0

counter=0


for x in range(0,len(df1)):
    if(df1['VOLUME1_PCE'][x]==df1['VOLUME2_BOX'][x]):
        df1['EQUAL_VOLUMES'][x]=1
    else:
        df1['EQUAL_VOLUMES'][x]=0

        
    if(df1['PER_PCE_NET_WEIGHT'][x]>=0.01 and df1['PER_PCE_NET_WEIGHT'][x]<=0.03):
        df1['PCE_NET_WEIGHT_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('PCE')
        
    if(df1['PER_BOX_NET_WEIGHT'][x]>=0.01 and df1['PER_BOX_NET_WEIGHT'][x]<=0.03):
        df1['BOX_NET_WEIGHT_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('BOX')


#RANGE: 0.01 TO 0.1 - THIS IS FOR THE GROSS WEIGHT

    if(df1['PER_PCE_GROSS_WEIGHT'][x]>=0.01 and df1['PER_PCE_GROSS_WEIGHT'][x]<=0.1):
        df1['PCE_GROSS_WEIGHT_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('PCE')
        
    if(df1['PER_BOX_GROSS_WEIGHT'][x]>=0.01 and df1['PER_BOX_GROSS_WEIGHT'][x]<=0.1):
        df1['BOX_GROSS_WEIGHT_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('BOX')
   
    
#WHATEVER IS THE MAJORITY THAT WILL BE THE QUANTITY UNIT 

#PRICE CHECK - 0.7 TO 12 USD

    if(df1['PER_PCE_UNIT_PRICE'][x]>=0.7 and df1['PER_PCE_UNIT_PRICE'][x]<=12):
        df1['PCE_PRICE_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('PCE')
        
    if(df1['PER_BOX_UNIT_PRICE'][x]>=0.7 and df1['PER_BOX_UNIT_PRICE'][x]<=12):
        df1['BOX_PRICE_CHECK'][x]='1'
        counter=counter+1
        df1['NEW_QUANTITY_UNIT'][x].append('BOX')
        
    counter=0

Creating Boolean checks for Net weight, Gross weight and Unit price - by Volume1 and Volume 2


In [33]:
df1['NEW_QUANTITY_UNIT'].tail(5)

1807    [PCE, BOX, PCE, BOX, PCE, BOX]
1808                                []
1809                                []
1810              [PCE, BOX, PCE, BOX]
1811                                []
Name: NEW_QUANTITY_UNIT, dtype: object

# DATA QUALITY INDEX 

<b>When the Volume 1 equals Volume 2 </b>


In [34]:
print('Assigning correct quantity unit - PCE/BOX')
df1['DQI']=''
df1['QUANTITY_UNIT_FINAL']=np.nan


df1['BOX_NET_WEIGHT_CHECK']=df1['BOX_NET_WEIGHT_CHECK'].astype(float)
df1['BOX_GROSS_WEIGHT_CHECK']=df1['BOX_GROSS_WEIGHT_CHECK'].astype(float)
df1['BOX_PRICE_CHECK']=df1['BOX_PRICE_CHECK'].astype(float)
df1['PCE_NET_WEIGHT_CHECK']=df1['PCE_NET_WEIGHT_CHECK'].astype(float)
df1['PCE_GROSS_WEIGHT_CHECK']=df1['PCE_GROSS_WEIGHT_CHECK'].astype(float)
df1['PCE_PRICE_CHECK']=df1['PCE_PRICE_CHECK'].astype(float)

df1['PCE_COUNT_IGNORE']=0


for x in range(0,len(df1['EQUAL_VOLUMES'])):
    if(df1['EQUAL_VOLUMES'][x]==1):
        df1['PCE_COUNT_IGNORE'][x]=df1['PCE_NET_WEIGHT_CHECK'][x]+df1['PCE_NET_WEIGHT_CHECK'][x]+df1['PCE_PRICE_CHECK'][x]
        
       # print('EQUAL VOLUME: ',df1['EQUAL_VOLUMES'][x])
       # print('net: ',df1['PCE_NET_WEIGHT_CHECK'][x])
       # print('gross: ',df1['PCE_GROSS_WEIGHT_CHECK'][x])
       # print('price',df1['PCE_PRICE_CHECK'][x])
        df1['DQI'][x]=int(df1['PCE_NET_WEIGHT_CHECK'][x])+int(df1['PCE_GROSS_WEIGHT_CHECK'][x])+int(df1['PCE_PRICE_CHECK'][x])
       # print('dqi: ',df1['DQI'][x])
        if(df1['DQI'][x]!=0):
            df1['QUANTITY_UNIT_FINAL'][x]='PCE'
        elif(df1['DQI'][x]==0):
            df1['DQI'][x]=0
            df1['QUANTITY_UNIT_FINAL'][x]=''
            df1['REQUIRES_MANUAL_VERIFICATION'][x]=1

#WILL NEED MANUAL VERIFICATION FOR THE EMPTY ONE HERE......
#There will be cases when the volumes are equal but we do not have any 

#CHECK THIS...
#df1['PCE_NET_WEIGHT_CHECK']=df1['PCE_NET_WEIGHT_CHECK'].fillna(0)
#df1['PCE_GROSS_WEIGHT_CHECK']=df1['PCE_GROSS_WEIGHT_CHECK'].fillna(0)

#df1['BOX_NET_WEIGHT_CHECK']=df1['BOX_NET_WEIGHT_CHECK'].fillna(0)
#df1['BOX_GROSS_WEIGHT_CHECK']=df1['BOX_GROSS_WEIGHT_CHECK'].fillna(0)

#df1['PCE_PRICE_CHECK']=df1['PCE_PRICE_CHECK'].fillna(0)
#df1['BOX_PRICE_CHECK']=df1['BOX_PRICE_CHECK'].fillna(0)

#df1['DQI_COUNTER']=np.nan


Assigning correct quantity unit - PCE/BOX


In [35]:
df1['DQI'].head(5)

0     
1    0
2    1
3    0
4     
Name: DQI, dtype: object

<b>When the Volume 1 and volume 2 are not equal </b> <br>
1) Find the quantity unit 


In [36]:

df1['BOX_COUNT_IGNORE']=0
print(df1['BOX_COUNT_IGNORE'].dtype)



for x in range(0,len(df1['EQUAL_VOLUMES'])):
    if(df1['EQUAL_VOLUMES'][x]==0):
        df1['PCE_COUNT_IGNORE'][x]=df1['PCE_NET_WEIGHT_CHECK'][x]+df1['PCE_NET_WEIGHT_CHECK'][x]+df1['PCE_PRICE_CHECK'][x]
        
        df1['BOX_COUNT_IGNORE'][x]=df1['BOX_NET_WEIGHT_CHECK'][x]+df1['BOX_GROSS_WEIGHT_CHECK'][x]+df1['BOX_PRICE_CHECK'][x]
        
        if (df1['PCE_COUNT_IGNORE'][x]>df1['BOX_COUNT_IGNORE'][x]):
            df1['DQI'][x]=df1['PCE_COUNT_IGNORE'][x]
            df1['QUANTITY_UNIT_FINAL'][x]='PCE'
        
        elif (df1['BOX_COUNT_IGNORE'][x] > df1['PCE_COUNT_IGNORE'][x]):
            df1['DQI'][x]=df1['BOX_COUNT_IGNORE'][x]
            df1['QUANTITY_UNIT_FINAL'][x]='BOX'
            
        elif (df1['PCE_COUNT_IGNORE'][x]==df1['BOX_COUNT_IGNORE'][x]):
            df1['DQI'][x]=0
            df1['QUANTITY_UNIT_FINAL'][x]='' 
            df1['REQUIRES_MANUAL_VERIFICATION'][x]=1

            
            
            

print(df1['BOX_NET_WEIGHT_CHECK'].dtype)
print(df1['PCE_NET_WEIGHT_CHECK'].dtype)
print(df1['PCE_PRICE_CHECK'].dtype)
#NEED THE ADDITION OF MRL HERE...

int64
float64
float64
float64


Based on this thought process, the proposed scale for Data Quality Index could be: <br>
100%- columns of the product import transaction were logical and the price pertest or box was cross-checked against MRPL (maintaining the rows you’ve already marked as 100%) <br>
75%- columns of the product import transaction were logical and the calculations made sense. We’re highly confident in this data but pricing info on MRPL was not available. <br>
50% - columns of the product import transaction were generally logical but 1 field (aka column) didn’t make sense and therefore decreases our confidence in the validity of the TA transaction row <br>
25%- columns of the product import transaction were somewhat logical, but had 2 fields (columns) that didn’t make sense so we really aren’t all that confident about the validity of the transaction row <br>
0%- Columns of the product import transaction had 3 or more fields that didn’t make sense so we aren’t confident in the transaction row <br>

In [37]:
df1['DQI'].head(5)

0    0
1    0
2    1
3    0
4    0
Name: DQI, dtype: object

# Filtration of RDT AND NON RDT

Now select "SARS-CoV-2 Antigen Rapid Diagnostic Tests" and "SARS-CoV-2 Antibody Rapid Diagnostic Tests" from Test Types

In [38]:
print('Filtering RDT tests')
df1=df1[(df1['Test_Type']=='SARS-CoV-2 Antigen Rapid Diagnostic Tests') | (df1['Test_Type']=='SARS-CoV-2 Antibody Rapid Diagnostic Tests')]

Filtering RDT tests


In [39]:
df1=df1.reset_index()

# SAVING THE FILE

In [40]:
#df1.to_csv('Accuracy_Number_Of_Test2.csv', index=False)


In [41]:
#r = re.compile(r'([0-9]+X[0-9]+T+|[0-9]+\*[0-9]+T|[0-9]+\*[0-9]+ T|[0-9]+X[0-9]+ T+)')
#ANSWER=r.search('YHLO IFLASH SARS-COV-2 IGM DUS,KIT 2*50 TES')
#if(ANSWER):
#    print('yes')


In [42]:
#using regex to identify Values like 2*50 and 2X50
#r = re.compile(r'([0-9]+X[0-9]+|[0-9]+\*[0-9])')

#for x in range(0,len(df1)):
#    if(r.search(df1['PRODUCT DETAILS'][x])):
#        print('True - Index: ',x)

# REFERENCE TO MRL

In [43]:
#READ THE MRL FILE 
print('Reading MRL File')
mrl=pd.read_excel("C:/Users/DrC/Desktop/Eureka Idea Co/Number of Test Testing File/Master reference list V2 CD.xlsx",sheet_name='Master Sheet 3.0-Deduped',header=1)
mrl.head(5)
list(mrl)

Reading MRL File


['Source: GF',
 'Source: Path',
 'FIND- Antigens (Dev/Under Eval)',
 'Type of Test',
 'Unique Identifier',
 'Merged Unique ID',
 'Manufacturer Product Catalogue number',
 'Product Name\n(IVD product)',
 'Product name- Special characters removed',
 'product_keywords',
 'Reference detail',
 'Manufacturer',
 'manufacturer_keywords',
 'Manufacturer HQ location- PATH',
 'Manufactuer location- ADP ',
 'Manufacturer HQ location- FIND',
 'Consolidated HQ Manufacturer Location (PATH Manufactuer HQ & ADP Manufacturer HQ)',
 'Phase ',
 'SRA Approval',
 'Total SRA Criteria Eligibility ',
 'Comments ',
 'EXPIRED ',
 'CE-IVD ',
 'US FDA EUA (US Food and Drug Administration Emergency Use Authorization) ',
 'EUA expiry?',
 'Health Canada/Interim Order',
 'WHO EUL (Emergency Use Listing)',
 'PMDA (Pharmaceuticals and medical devices agency)',
 'TGA (Therapeutic Goods Administration)',
 'IFU Link',
 'IFU Link ',
 'IFU Link (Research)',
 ' IFU link Combined sources [Info from Column P&Q] (IFU links from 

In [44]:
#df1_mrl = pd.merge(df1, mrl, left_on='PRODUCT DETAILS',right_on='Product Name\n(IVD product)',how='left')

In [45]:
#df1_mrl['Product Name\n(IVD product)'].unique()

In [46]:
#for x in range(0,len(df1_mrl)):
#    if(df1_mrl['PRODUCT DETAILS'][x]==df1_mrl['Product Name\n(IVD product)'][x]):
#        df1_mrl['UNIQUE_ID'][x]=df1_mrl['Unique Identifier'][x]

In [47]:
#df1_mrl['UNIQUE_ID'].unique()

# Cleaning the PRODUCT DETAILS FOR MATCHING

In [48]:
print('Preparing product details in MRL for matching')
df1['PDC_NEW_DF']=df1['PRODUCT DETAILS']

Preparing product details in MRL for matching


In [49]:
df1['PDC_NEW_DF']= df1['PDC_NEW_DF'].str.strip() 
df1['PDC_NEW_DF'] = df1['PDC_NEW_DF'].str.replace(r'[^\w\s]+', '') 
#Removing double spaces between the words
df1['PDC_NEW_DF']= df1['PDC_NEW_DF'].replace('\s+', ' ', regex=True)
df1['PDC_NEW_DF']=df1['PDC_NEW_DF'].str.upper()



# Cleaning the Product Name\n(IVD product FROM MRL FOR MATCHING
                            

In [50]:
mrl['PDC_NEW_MRL']=mrl['Product Name\n(IVD product)']

In [51]:
mrl['PDC_NEW_MRL']= mrl['PDC_NEW_MRL'].str.strip() 
mrl['PDC_NEW_MRL'] = mrl['PDC_NEW_MRL'].str.replace(r'[^\w\s]+', '') 
#Removing double spaces between the words
mrl['PDC_NEW_MRL']= mrl['PDC_NEW_MRL'].replace('\s+', ' ', regex=True)
mrl['PDC_NEW_MRL']=mrl['PDC_NEW_MRL'].str.upper()

In [52]:
# https://www.statology.org/fuzzy-matching-pandas/

print('Creating a column with close matches between product details in Trade Atlas and MRL.\nThis step shall take a while. Please be patient.')
import difflib 

from difflib import SequenceMatcher

mrl['product_match'] = mrl['PDC_NEW_MRL']

#convert product_name_mrl in mrl to product name it most closely matches in df1

mrl['PDC_NEW_MRL'] = mrl['PDC_NEW_MRL'].apply(lambda x: (difflib.get_close_matches(x, df1['PDC_NEW_DF'])[:1] or [None])[0])
#df2['Unit'] = df2['Unit'].apply(lambda x: (difflib.get_close_matches(x, df1['Unit'])[:1] or [None])[0])

#https://stackoverflow.com/questions/36557722/python-pandas-difflib-throws-list-index-out-of-range-error

#convert team name in df2 to team name it most closely matches in df1
print('Matches completed')

Creating a column with close matches between product details in Trade Atlas and MRL.
This step shall take a while. Please be patient.
Matches completed


In [53]:
mrl['PDC_NEW_MRL']

#merge the DataFrames into one
#new = df1.merge(mrl)
#view final DataFrame
#print(new)

0                                               None
1                                               None
2                SARCOV2 AG RAPID TEST CASSETTE BAIK
3           SARSCOV2 ANTIGEN TEST KIT COLLOIDAL GOLD
4       SARSCOV2 ANTIGEN SALIVA RAPID TEST KIT 1TBOX
                            ...                     
1376                                            None
1377                         HUMASIS COVID19 AG TEST
1378                         LIAISON SARSCOVS1S2 IGG
1379             RAPID SARSCOV2 ANTIBODY IGMIGG TEST
1380    PREVENT COVID 19 ANTIGEN SWAB NASOPHARINGEAL
Name: PDC_NEW_MRL, Length: 1381, dtype: object

In [54]:
len(df1)

745

In [55]:
#Creating a key in df1
df1['MERGE_KEY']=''

for x in range(0,len(df1)):
    df1['MERGE_KEY'][x]=x

In [56]:
#df1.to_csv('check4.csv', index=False)

In [57]:
df1_mrl = pd.merge(df1, mrl, left_on='PDC_NEW_DF',right_on='PDC_NEW_MRL',how='left')

In [58]:
#df1_mrl[['PDC_NEW_MRL', 'PDC_NEW_DF']].tail(5)

In [59]:
df1_mrl['PDC_NEW_DF']=df1_mrl['PDC_NEW_DF'].astype(str)
df1_mrl['PDC_NEW_MRL']=df1_mrl['PDC_NEW_MRL'].astype(str)
df1_mrl['product_match']=df1_mrl['product_match'].astype(str)


In [60]:
print('Analyzing the match percentage between Product detail (Trade Atlas) and Product Details (MRL)...')
from difflib import SequenceMatcher
#df1['MATCH'] = np.empty((len(df1), 0)).tolist()
df1_mrl['MATCH_PERCENT']=''
for x in range(0,len(df1_mrl)):
    df1_mrl['MATCH_PERCENT'][x]= SequenceMatcher(None, df1_mrl['PDC_NEW_DF'][x], df1_mrl['product_match'][x]).ratio()


Analyzing the match percentage between Product detail (Trade Atlas) and Product Details (MRL)...


In [61]:
#df1_mrl.to_csv('Cross_reference.csv', index=False)

In [62]:
#df1_mrl['MATCH_PERCENT']
#df1_mrl[['MERGE_KEY','MATCH_PERCENT']]

In [63]:
#df1_mrl['MERGE_KEY'].nunique()

In [64]:
grouped_df=df1_mrl.groupby(['MERGE_KEY'])['MATCH_PERCENT'].max().reset_index() #finding the index and maximum value

In [65]:
grouped_df=pd.merge(df1_mrl, grouped_df, on=['MERGE_KEY', 'MATCH_PERCENT']) #By Default it is inner join.

In [66]:
#len(grouped_df)


#If a match is found, then put the MRPL Unique ID in Column MRL Unique ID (BP). 
#Need to be more careful about searching the product names, sometimes, there is not an exact match
#(Matching every character) with the products in the MRL. 
#We need to search a part of the name or match the Exporter's name in TA with the Manufacturer's name in MRL.


#MATCH THE EXPORTERS NAME WITH THE MANUFACTURER NAME


# MRL CHECK BOOLEAN 

In [67]:
print('Creating MRL Boolean Check')
grouped_df['MRL_CHECK']=0
for x in range(0,len(grouped_df)):
    if(grouped_df['MATCH_PERCENT'][x]>=0.70):
        grouped_df['MRL_CHECK'][x]=1
    else:
        grouped_df['MRL_CHECK'][x]=0

Creating MRL Boolean Check


In [68]:
#Now these are not duplicates. It is possible that a product in the trade atlas matches with two products in the MRL with the same 
#Match percent

In [69]:
#list(grouped_df)

# DEFINING THE DQI

In [70]:
print('Analyzing the Data Quality Index')
for x in range(0,len(grouped_df)):
    grouped_df['DQI'][x]=grouped_df['PCE_NET_WEIGHT_CHECK'][x]+grouped_df['PCE_GROSS_WEIGHT_CHECK'][x]+grouped_df['PCE_PRICE_CHECK'][x] +grouped_df['MRL_CHECK'][x]
    

grouped_df['DQI_FINAL']=''    

for x in range(0,len(grouped_df)):
    if(grouped_df['DQI'][x]==0):
        grouped_df['DQI_FINAL'][x]= '0'
    elif(grouped_df['DQI'][x]==1):
        grouped_df['DQI_FINAL'][x]= '25'
    elif(grouped_df['DQI'][x]==2):
        grouped_df['DQI_FINAL'][x]= '50'
    elif(grouped_df['DQI'][x]==3):
        grouped_df['DQI_FINAL'][x]= '75'
    elif(grouped_df['DQI'][x]==4):
        grouped_df['DQI_FINAL'][x]= '100'

        


Analyzing the Data Quality Index


In [71]:
#grouped_df['DQI_FINAL']

# NEW THINGS THAT NEEDS TO BE DONE 

<b>if there is a match between product detail and MRL then pull the MRL UNIQUE NUMBER - DONE</b>

In [72]:
print('Pulling MRL Unique Number')
grouped_df['UNIQUE_ID_PULL']=''
for x in range(0,len(grouped_df)):
    if(grouped_df['MRL_CHECK'][x]==1):
        grouped_df['UNIQUE_ID_PULL'][x]=grouped_df['Unique Identifier'][x]
    
#list(mrl)

Pulling MRL Unique Number


In [73]:
#If we get a match in MRL for the product, then need to match the unit price (if it is available in MRL). If we get a similar unit price, then we can say 100% Confidence.

print('Boolean check for Unit price match between MRL Unit Price and calculated Unit price')
#grouped_df['Global Fund-Reference price per pack EXW, USD']=grouped_df['Global Fund-Reference price per pack EXW, USD'].round(2)
#grouped_df['ADP- Indicative purchase pricing (per test)']=grouped_df['ADP- Indicative purchase pricing (per test)'].round(2)
#grouped_df['Global Fund- Reference price per test EXW, USD']=grouped_df['Global Fund- Reference price per test EXW, USD'].round(2)
#grouped_df['Consolidated price per test (Global Fund- Reference price per test + ADP- Indicative purchase pricing (per test)']=grouped_df['Consolidated price per test (Global Fund- Reference price per test + ADP- Indicative purchase pricing (per test)'].round(2)

#If there is a match and if any of the value matches the price, then I say we have a match. This deals with the different pricing issue. Different pricing makes the MRL list invalid initself.
grouped_df['UNIT_PRICE_MATCH']=''
for x in range(0,len(grouped_df)):
    if(grouped_df['QUANTITY_UNIT_FINAL'][x]=='PCE'):
        if((grouped_df['PER_PCE_UNIT_PRICE'][x]==grouped_df['Global Fund-Reference price per pack EXW, USD'][x]) 
          | (grouped_df['PER_PCE_UNIT_PRICE'][x]==grouped_df['ADP- Indicative purchase pricing (per test)'][x]) | (grouped_df['PER_PCE_UNIT_PRICE'][x]==grouped_df['Global Fund- Reference price per test EXW, USD'][x]) |
          (grouped_df['PER_PCE_UNIT_PRICE'][x]==grouped_df['Consolidated price per test (Global Fund- Reference price per test + ADP- Indicative purchase pricing (per test)'][x])):
            grouped_df['UNIT_PRICE_MATCH'][x]=1
            
    if(grouped_df['QUANTITY_UNIT_FINAL'][x]=='BOX'):
        if((grouped_df['PER_BOX_UNIT_PRICE'][x]==grouped_df['Global Fund-Reference price per pack EXW, USD'][x]) 
        | (grouped_df['PER_BOX_UNIT_PRICE'][x]==grouped_df['ADP- Indicative purchase pricing (per test)'][x]) | (grouped_df['PER_BOX_UNIT_PRICE'][x]==grouped_df['Global Fund- Reference price per test EXW, USD'][x]) |
        (grouped_df['PER_BOX_UNIT_PRICE'][x]==grouped_df['Consolidated price per test (Global Fund- Reference price per test + ADP- Indicative purchase pricing (per test)'][x])):
            grouped_df['UNIT_PRICE_MATCH'][x]=1


#The prices are empty for the majority of the rows in the MRL. should this step be included? 

# 'PER_PCE_UNIT_PRICE'



Boolean check for Unit price match between MRL Unit Price and calculated Unit price


In [74]:
#grouped_df['Global Fund-Reference price per pack EXW, USD'].value_counts()
#grouped_df['ADP- Indicative purchase pricing (per test)'].value_counts()

In [75]:
#Create a new column, modification of reference detail.
print(' Performing Boolean check for Number of tests match between MRL and Trade Atlas ')

r = re.compile(r'([0-9]+X[0-9]+T+|[0-9]+\*[0-9]+T|[0-9]+\*[0-9]+ T|[0-9]+X[0-9]+ T+)')
grouped_df['REFERENCE_DETAIL_CLEAN']=''
grouped_df['REFERENCE_DETAIL_CLEAN']=grouped_df['REFERENCE_DETAIL_CLEAN'].astype(str)
grouped_df['Reference detail']=grouped_df['Reference detail'].astype(str)
grouped_df['REFERENCE_DETAIL_MATCH']=''

#1) If there is a patter like x then store that value

for x in range(0,len(grouped_df)):
    if(r.search(grouped_df['Reference detail'][x])):
        grouped_df['REFERENCE_DETAIL_CLEAN'][x]='' #find all gets the number as it is 
    else:
        grouped_df['REFERENCE_DETAIL_CLEAN'][x]=re.sub("[^0-9]","", grouped_df['Reference detail'][x])



        
#r.findall(grouped_df['Reference detail'][x])  THIS IS IMPORTANT AS IT GETS THE VALUE FOR US        
#This below code is a problem but is important
#elif(int(re.sub("[^0-9]", "", grouped_df['Reference detail'][x]))>=5 and int(re.sub("[^0-9]", "", grouped_df['Reference detail'][x]))<=2500):


 Performing Boolean check for Number of tests match between MRL and Trade Atlas 


<b>PROBLEM IDENTIFIED: WHAT TO DO FOR REFERENCE DETAIL WITH THE VALUE: 24T/kit 48T/kit 96T/kit </b>


In [76]:
grouped_df['REFERENCE_DETAIL_CLEAN']=grouped_df['REFERENCE_DETAIL_CLEAN'].replace('',np.nan)

In [77]:
#print(grouped_df['REFERENCE_DETAIL_CLEAN'].unique())

In [78]:
#Compare the quantity with the extracted number of test.
#if the quantity is equal then Reference Detail Match = 1 
grouped_df['REFERENCE_DETAIL_CLEAN']=grouped_df['REFERENCE_DETAIL_CLEAN'].astype(float)
#matching reference detail clean with the the test quantity final..         
for x in range(0,len(grouped_df)):
    if(grouped_df['test_quantity_final'][x]==grouped_df['REFERENCE_DETAIL_CLEAN'][x]):
        grouped_df['REFERENCE_DETAIL_MATCH'][x]=1
        
    else:
        grouped_df['REFERENCE_DETAIL_MATCH'][x]=0


In [79]:
#grouped_df[['test_quantity_final','REFERENCE_DETAIL_CLEAN','REFERENCE_DETAIL_MATCH']].loc[125:135,]

<b>if there are two matches then select the one whose Manufacturer matches with the exporter name</b>

In [80]:
#if there is a duplicate then I have set the 

# Removing duplicates 

In [81]:
print('Removing duplicates in the Trade Atlas and MRL product match')
grouped_df=grouped_df.drop_duplicates(subset='MERGE_KEY').reset_index(drop=True) #removing the duplicated IDS.

Removing duplicates in the Trade Atlas and MRL product match


In [82]:
print('Exporting file')
grouped_df.to_csv('OUTPUT1.csv', index=False)

Exporting file


In [83]:
print('completed')

completed
